<hi>An analysis of global deforestation data between 1990 - 2016.</h1>

Task:

ForestQuery is on a mission to combat deforestation around the world and to raise awareness about this topic and its impact on the environment. The data analysis team at ForestQuery has obtained data from the World Bank that includes forest area and total land area by country and year from 1990 to 2016, as well as a table of countries and the regions to which they belong.
The data analysis team has used SQL to bring these tables together and to query them in an effort to find areas of concern as well as areas that present an opportunity to learn from successes.


In [10]:
#pip install pandasql #install the querying library

import pandas as pd
from pandasql import sqldf

In [13]:
#For ease of querying using a python querying library, a view "deforestation" has been created from the tables

"""
CREATE VIEW deforestation AS
SELECT *
FROM (
SELECT r.region ,f.country_name
,f.year
,f.forest_area_sqkm
,l.total_area_sq_mi * 2.59 AS total_area_sqkm
,(f.forest_area_sqkm / (l.total_area_sq_mi * 2.59)) * 100 AS forest_percentage ,income_group
FROM land_area l
FULL JOIN forest_area f ON l.country_code = f.country_code
AND l.year = f.year
FULL JOIN regions r ON f.country_code = r.country_code ORDER BY 2
,3 ) t1
"""

'\nCREATE VIEW deforestation AS\nSELECT *\nFROM (\nSELECT r.region ,f.country_name\n,f.year\n,f.forest_area_sqkm\n,l.total_area_sq_mi * 2.59 AS total_area_sqkm\n,(f.forest_area_sqkm / (l.total_area_sq_mi * 2.59)) * 100 AS forest_percentage ,income_group\nFROM land_area l\nFULL JOIN forest_area f ON l.country_code = f.country_code\nAND l.year = f.year\nFULL JOIN regions r ON f.country_code = r.country_code ORDER BY 2\n,3 ) t1\n'

In [12]:
deforestation = pd.read_csv('deforestation.csv')
deforestation.head()

,region,country_name,year,forest_area_sqkm,total_area_sqkm,forest_percentage,income_group
0,South Asia,Afghanistan,1990.0,13500.0,652860.005,2.067825,Low income
1,South Asia,Afghanistan,1991.0,13500.0,652860.005,2.067825,Low income
2,South Asia,Afghanistan,1992.0,13500.0,652860.005,2.067825,Low income
3,South Asia,Afghanistan,1993.0,13500.0,652860.005,2.067825,Low income
4,South Asia,Afghanistan,1994.0,13500.0,652860.005,2.067825,Low income


In [14]:
#querying the database using python gives the added functionality of being able to analyze the dataset with both Python and SQL
deforestation.describe() #description of the fields in the deforestation data

,year,forest_area_sqkm,total_area_sqkm,forest_percentage
count,5886.000000,5.570000e+03,5.808000e+03,5534.000000
mean,2003.000000,3.910518e+05,1.183877e+06,32.757533
std,7.789543,2.901344e+06,8.799095e+06,24.059251
min,1990.000000,8.000000e-01,1.994300e+00,0.000536
25%,1996.000000,1.676075e+03,1.012001e+04,11.502315
50%,2003.000000,2.051300e+04,9.150001e+04,31.749618
75%,2010.000000,9.901500e+04,4.463000e+05,50.185798
max,2016.000000,4.128269e+07,1.273809e+08,98.910257


Task: According to the worldbank, the loss in forest area between 2016 and 1990 was 1,324,449.0 sq km, this figure is slightly over the entire land area of a country, what is that country?

In [34]:
query = """
SELECT country_name ,total_area_sqkm
FROM (
SELECT *
FROM deforestation WHERE year = 2016 ) t1
WHERE total_area_sqkm < 1324449 ORDER BY 2 DESC
        
        """
sqldf(query)

,country_name,total_area_sqkm
0,Peru,1.280000e+06
1,Niger,1.266700e+06
2,Chad,1.259200e+06
3,Angola,1.246700e+06
4,Mali,1.220190e+06
...,...,...
193,"Macao SAR, China",3.040660e+01
194,Tuvalu,2.999220e+01
195,Nauru,1.999480e+01
196,Gibraltar,9.997400e+00


Task: categorize the regioon in descending order of percentage allocation of forest areas

In [36]:
query = """
SELECT region
,sum(forest_area_sqkm) AS forest
,sum(total_area_sqkm) AS total_land
,(sum(forest_area_sqkm) / sum(total_area_sqkm)) * 100 AS forest_percent
FROM deforestation WHERE year = 2016
AND region != 'World' GROUP BY 1
ORDER BY 4 DESC
        
        """
sqldf(query)

,region,forest,total_land,forest_percent
0,Latin America & Caribbean,9.250586e+06,2.003936e+07,46.162072
1,Europe & Central Asia,1.043861e+07,2.744011e+07,38.041422
2,North America,6.573934e+06,1.824098e+07,36.039361
3,Sub-Saharan Africa,6.115291e+06,2.124236e+07,28.788188
4,East Asia & Pacific,6.421326e+06,2.436134e+07,26.358677
5,South Asia,8.353105e+05,4.771604e+06,17.505863
6,Middle East & North Africa,2.321310e+05,1.122347e+07,2.068265


Task: Which countries saw the biggest change in land allocated to forestation between 1990 and 2016?

Approach:
A casestudy comparing the forest data between 1990 and 2016 was evaluated to see the countries that saw the biggest change in land area. Two methods were used:
i. forest area difference
ii. percentage change in forest area

In [37]:
#query showing country rank by differene in land masss allocated to forestation between 2016 and 2011
query = """
        SELECT country_name
        ,forest_area_sqkm AS forest_2016
        ,forest_area_1990
        ,forest_area_sqkm - forest_area_1990 AS forest_change ,((forest_area_sqkm - forest_area_1990) / forest_area_1990) * 100 AS
        percentage_forest_change FROM (
        SELECT country_name ,year
        ,(forest_area_sqkm) ,lag(forest_area_sqkm) OVER (
        PARTITION BY country_name ORDER BY year
        ) forest_area_1990 FROM deforestation
        WHERE year = 1990 OR year = 2016
        AND forest_percentage IS NOT NULL ORDER BY 1
        ,2 ) t1
        WHERE (forest_area_sqkm - forest_area_1990 / forest_area_1990) * 100 IS NOT NULL AND country_name != 'World'
        ORDER BY 4 DESC
        
        """
sqldf(query)

,country_name,forest_2016,forest_area_1990,forest_change,percentage_forest_change
0,China,2.098635e+06,1571405.938,527229.06200,33.551424
1,United States,3.103700e+06,3024500.000,79200.00000,2.618615
2,India,7.086040e+05,639390.000,69213.98440,10.825003
3,Russian Federation,8.148895e+06,8089500.000,59395.00000,0.734223
4,Vietnam,1.490200e+05,93630.000,55390.00000,59.158389
...,...,...,...,...,...
198,Tanzania,4.568800e+05,559200.000,-102320.00000,-18.297568
199,Nigeria,6.583400e+04,172340.000,-106506.00098,-61.799931
200,Myanmar,2.849460e+05,392180.000,-107234.00390,-27.343058
201,Indonesia,9.032560e+05,1185450.000,-282193.98440,-23.804799


In [38]:
#query showing top percentage decrease in percentage of forest area allocated
query = """
        SELECT country_name ,forest_2016
        ,forest_area_1990
        ,abs(forest_change) AS forest_change ,abs(percentage_forest_change) AS PERCENT
        FROM (
        SELECT country_name
        ,forest_area_sqkm AS forest_2016
        ,forest_area_1990
        ,forest_area_sqkm - forest_area_1990 AS forest_change ,((forest_area_sqkm - forest_area_1990) / forest_area_1990) * 100 AS
        percentage_forest_change FROM (
        SELECT country_name ,year
        ,(forest_area_sqkm)
        ,lag(forest_area_sqkm) OVER (
        PARTITION BY country_name ORDER BY year ) forest_area_1990
        FROM deforestation WHERE year = 1990
        OR year = 2016
        AND forest_percentage IS NOT NULL ORDER BY 1
        ,2 ) t1
        WHERE (forest_area_sqkm - forest_area_1990 / forest_area_1990) * 100 IS NOT NULL ORDER BY 5 DESC
        ) t2
        WHERE forest_change < 0
        AND country_name != 'World'
        ORDER BY 5 DESC
        
        """
sqldf(query)

,country_name,forest_2016,forest_area_1990,forest_change,PERCENT
0,Togo,1.682000e+03,6850.0,5168.000031,75.445256
1,Nigeria,6.583400e+04,172340.0,106506.000980,61.799931
2,Uganda,1.941800e+04,47510.0,28091.999510,59.128603
3,Mauritania,2.210000e+03,4150.0,1940.000000,46.746988
4,Honduras,4.472000e+04,81360.0,36640.000000,45.034415
...,...,...,...,...,...
83,Canada,3.470224e+06,3482730.0,12505.937000,0.359084
84,Luxembourg,8.670000e+02,870.0,3.000030,0.344831
85,Sierra Leone,3.107600e+04,31180.0,103.999020,0.333544
86,Papua New Guinea,3.355620e+05,336270.0,708.007800,0.210547
